---
title: Atlas de Ciudades
subtitle: Descargando datos del portal de ONU Habitat
---

ONU Habitat publicó un portal que reune datos urbanos de Bolivia. En este cuaderno descargo toda la información que puedo de él.

# Mapas con datos del Censo 2012

Primero construyo un índice de [los mapas que existen](https://datosurbanosdebolivia.abe.bo/atlas_ciudades) y selecciono aquellos que incluyen datos del Censo de 2012.

In [152]:
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup
import requests
from urllib.parse import quote_plus
from tqdm import tqdm

In [ ]:
def crear_indice_capas():
    """
    Consulta el índice de mapas, extrae atributos relevantes,
    particularmente el nombre de cada capa.
    """
    
    seleccion = [
        'Población',
        'Tasa de dependencia Demográfica',
        'Tasa de dependencia Económica',
        'Tasa global de participación',
        'Predominancia de actividades económicas',
        '% Migrante reciente',
        '% Casados / as',
        'Edad mediana',
        'Años de estudio',
        '% Vivienda Alquilada',
        '% Deficit Cualitativo en Vivienda',
        'Personas por dormitorio',
        '% pobreza por NBI'
    ]

    url = 'https://datosurbanosdebolivia.abe.bo/api/cpi/ind_cpi/listar_coberturas_atlas'
    response = requests.get(mapas_indice_url)

    indice = []
    
    for i in response.json()['data']:
        item = {
            'nombre': i['aco_nombre'], 
            'capa': i['aco_wms_layers_name'], 
            'descripcion': i['aco_definicion']
        }
        indice.append(item)
    
    df = pd.DataFrame(indice)
    return df[df.nombre.isin(seleccion)]

In [74]:
indice_capas = crear_indice_capas()
indice_capas

,nombre,capa,descripcion
0,Población,pndic:Poblacion,El conjunto de personas que habitan en una det...
2,Tasa de dependencia Demográfica,pndic:TD_Demografia,Expresa la proporción existente entre la pobla...
3,Tasa de dependencia Económica,pndic:TD_economica,Indicador de la relación que existe entre la p...
4,Tasa global de participación,pndic:TGP,Indicador que mide la proporción de la poblaci...
5,Predominancia de actividades económicas,pndic:F_ocupados,Muestra el grado de especialización de las act...
6,% Migrante reciente,pndic:F_migra5,Muestra la proporción de la población que se h...
7,% Casados / as,pndic:F_casado,Muestra la proporción de la población que tien...
8,Edad mediana,pndic:EdadMedian,Indicador del grado de envejecimiento de la po...
9,Años de estudio,pndic:Anios_Estud,Indicador del promedio de años de escolaridad ...
10,% Vivienda Alquilada,pndic:F_Alquilada,Proporción de las viviendas en los que los hog...


Las unidades geográficas en estos mapas son iguales y en lugar de republicar cada capa individualmente, sería más eficiente tener una sóla capa con propiedades para cada valor. Primero descargo los valores de propiedades en cada capa sin ninguna geometría y construyo una tabla.

In [81]:
def identificar_propiedad(capa):
    """
    Descarga un sólo objeto de una capa para identificar el
    nombre de la propiedad relevante.
    """
    
    df = pd.read_csv(
            f'https://pndic-geoserver.abe.bo/geoserver/ows?service=WFS&request=GetFeature&typeName={capa}&outputFormat=csv&count=1'
        )
    
    return [col for col in df.columns if col not in ['FID', 'geom', 'id_unico', 'nombreciud', 'name']][0]

def descargar_capa(capa, nombre):
    """
    Descarga una capa como csv y sin geometrías.
    """
    
    propiedad = identificar_propiedad(capa)
    url = f'https://pndic-geoserver.abe.bo/geoserver/ows?service=WFS&request=GetFeature&typeName={capa}&outputFormat=csv&PropertyName={quote_plus(propiedad)},id_unico,nombreciud'
    df = pd.read_csv(url)
    return df.set_index(['id_unico','nombreciud'])[propiedad].rename(nombre)

In [79]:
capas = []
for i, row in indice_capas.iterrows():
    capas.append(descargar_capa(row['capa'], row['nombre']))
capas = pd.concat(capas, axis=1)

In [80]:
capas

,,Población,Tasa de dependencia Demográfica,Tasa de dependencia Económica,Tasa global de participación,Predominancia de actividades económicas,% Migrante reciente,% Casados / as,Edad mediana,Años de estudio,% Vivienda Alquilada,% Deficit Cualitativo en Vivienda,Personas por dormitorio,% pobreza por NBI
id_unico,nombreciud,,,,,,,,,,,,,
3018993543,Yacuiba,62,0.58974359,0.823529412,69.3877551,3,12.72727273,23.80952381,26,7.243243243,38.46153846,23.07692308,2.394505495,22.58
5494722258,Montero,34,1,1.428571429,63.63636364,3,7.142857143,47.36842105,19,10.11764706,40,90,1.5,0
5492266109,Montero,41,0.64,1.05,71.42857143,3,14.70588235,12,19,6.571428571,0,100,3.416666667,85.37
598960005,Viacha,24,0.2,3.8,23.80952381,1,0,40,20.5,6.384615385,0,42.85714286,2.285714286,79.17
5348027391,Warnes,106,0.630769231,1.078431373,64.55696203,3,9.677419355,26.08695652,20,10.32758621,0,20.83333333,2.529166667,19.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477188422,Oruro,228,0.452229299,1.303030303,54.21052632,3,7.804878049,37.35632184,24,11.85810811,26.08695652,20.28985507,2.04589372,SD
478054668,Oruro,6,2,2,50,3,0,50,11,11,0,100,6,SD
478356690,Oruro,87,0.553571429,1.12195122,60.86956522,3,7.594936709,33.89830508,22,9.723404255,20,45,3.020833333,SD


Y luego descargo todas las geometrías:

In [87]:
def descargar_geometrias(capa):
    
    url = f'https://pndic-geoserver.abe.bo/geoserver/ows?service=WFS&request=GetFeature&typeName={capa}&outputFormat=json&PropertyName=id_unico,nombreciud,geom'
    gdf = gpd.read_file(url)
    gdf.id_unico = gdf.id_unico.astype(int)
    return gdf.set_index(['id_unico','nombreciud']).geometry

In [88]:
geometrias = descargar_geometrias('pndic:Poblacion')

In [89]:
geometrias

id_unico    nombreciud
3018993543  Yacuiba       MULTIPOLYGON (((-63.68088 -22.00700, -63.68094...
5494722258  Montero       MULTIPOLYGON (((-63.24110 -17.35756, -63.24110...
5492266109  Montero       MULTIPOLYGON (((-63.28062 -17.35402, -63.28088...
598960005   Viacha        MULTIPOLYGON (((-68.29461 -16.65807, -68.29467...
5348027391  Warnes        MULTIPOLYGON (((-63.14458 -17.60526, -63.14399...
                                                ...                        
477188422   Oruro         MULTIPOLYGON (((-67.09902 -17.97215, -67.09903...
478054668   Oruro         MULTIPOLYGON (((-67.09005 -17.96066, -67.09012...
478356690   Oruro         MULTIPOLYGON (((-67.09717 -17.95689, -67.09718...
478011828   Oruro         MULTIPOLYGON (((-67.11147 -17.96113, -67.11175...
477046482   Oruro         MULTIPOLYGON (((-67.06863 -17.97408, -67.06864...
Name: geometry, Length: 107858, dtype: geometry

Como los índices en ambas tablas son iguales, las uno y creo una sola tabla con todas las propiedades y una columna de geometrías que puedo guardar en formato GeoJSON.

In [90]:
atlas = gpd.GeoDataFrame(pd.concat([capas, geometrias], axis=1), geometry='geometry')

In [91]:
atlas

,,Población,Tasa de dependencia Demográfica,Tasa de dependencia Económica,Tasa global de participación,Predominancia de actividades económicas,% Migrante reciente,% Casados / as,Edad mediana,Años de estudio,% Vivienda Alquilada,% Deficit Cualitativo en Vivienda,Personas por dormitorio,% pobreza por NBI,geometry
id_unico,nombreciud,,,,,,,,,,,,,,
3018993543,Yacuiba,62,0.58974359,0.823529412,69.3877551,3,12.72727273,23.80952381,26,7.243243243,38.46153846,23.07692308,2.394505495,22.58,"MULTIPOLYGON (((-63.68088 -22.00700, -63.68094..."
5494722258,Montero,34,1,1.428571429,63.63636364,3,7.142857143,47.36842105,19,10.11764706,40,90,1.5,0,"MULTIPOLYGON (((-63.24110 -17.35756, -63.24110..."
5492266109,Montero,41,0.64,1.05,71.42857143,3,14.70588235,12,19,6.571428571,0,100,3.416666667,85.37,"MULTIPOLYGON (((-63.28062 -17.35402, -63.28088..."
598960005,Viacha,24,0.2,3.8,23.80952381,1,0,40,20.5,6.384615385,0,42.85714286,2.285714286,79.17,"MULTIPOLYGON (((-68.29461 -16.65807, -68.29467..."
5348027391,Warnes,106,0.630769231,1.078431373,64.55696203,3,9.677419355,26.08695652,20,10.32758621,0,20.83333333,2.529166667,19.81,"MULTIPOLYGON (((-63.14458 -17.60526, -63.14399..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477188422,Oruro,228,0.452229299,1.303030303,54.21052632,3,7.804878049,37.35632184,24,11.85810811,26.08695652,20.28985507,2.04589372,SD,"MULTIPOLYGON (((-67.09902 -17.97215, -67.09903..."
478054668,Oruro,6,2,2,50,3,0,50,11,11,0,100,6,SD,"MULTIPOLYGON (((-67.09005 -17.96066, -67.09012..."
478356690,Oruro,87,0.553571429,1.12195122,60.86956522,3,7.594936709,33.89830508,22,9.723404255,20,45,3.020833333,SD,"MULTIPOLYGON (((-67.09717 -17.95689, -67.09718..."


In [92]:
atlas.to_file('datos/atlas.geojson')

# Indicadores urbanos

In [132]:
def descargar_indicador(indicador_codigo, indicador_nombre):
    
    url = f'https://datosurbanosdebolivia.abe.bo/api/pndic/ind_pndic/listar_todo_indicador_politica_ciudades/{indicador_codigo}'
    response = requests.get(url)
    
    ficha = response.json()['data']['indicador'][0]
    metadato = {
        'metodo': ficha['ind_metodo_calculo'],
        'fuente': ficha['ind_fuente'],
        'descripcion': ficha['ind_descripcion_detallada'],
        'resolucion': ficha['nig_nivel_geograficos'][0]['nig_descripcion']
    }
    
    variables = response.json()['data']['indicador_variable']
    datos = []
    
    for variable in variables:
        
        v = {
            'indicador': indicador_nombre,
            'indicador_codigo': indicador_codigo,
            'nombre': variable['inv_descripcion'],
            'unidad': variable['uni_descripcion'],
            'periodo': variable['ges_descripcion'],
            'variable_codigo': variable['inv_id']
        }
        
        if variable['mun_detalle']:
            
            serie = pd.DataFrame(variable['mun_detalle'])
            serie.columns = ['municipio_id', 'municipio', 'valor']
            for key in v.keys():
                serie.insert(0, key, v[key])
            datos.append(serie)
    
    datos = pd.concat(datos)
    
    return metadato, datos

In [133]:
url = 'https://datosurbanosdebolivia.abe.bo/api/pndic/ind_pndic/listar_todo_pndic'
response = requests.get(url)

indicadores = []
variables = []
for objetivo in response.json()['data']:
    for linea in objetivo['lin_linea_estrategicas']:
        for indicador in linea['ind_indicadores']:
            
            i = {
                'objetivo': objetivo['obj_politica_ciudades'],
                'objetivo_descripcion': objetivo['obj_informacion'],
                'linea': linea['lin_descripcion'],
                'indicador': indicador['ind_descripcion'],
                'indicador_codigo': indicador['ind_id']
            }
            
            metadato, datos = descargar_indicador(i['indicador_codigo'], i['indicador'])
            
            indicadores.append({**i, **metadato})
            variables.append(datos)

indicadores = pd.DataFrame(indicadores)
variables = pd.concat(variables)

El catálogo de indicadores urbanos

In [134]:
indicadores

,objetivo,objetivo_descripcion,linea,indicador,indicador_codigo,metodo,fuente,descripcion,resolucion
0,Ciudades inclusivas,Con el fin de lograr la integración de la ciud...,Disminuir las brechas de género y generacionales,Índice modificado de Inequidad de Género,2,Diferencias entre hombres y mujeres sobre: seg...,Encuesta de Hogares - EH 2019 del INE.,Se utilizaron las siguientes variables para Ho...,Departamental - Urbano
1,Ciudades inclusivas,Con el fin de lograr la integración de la ciud...,Reducir la desigualdad y la segregación socioe...,Índice de Inequidad socioespacial,3,"Porcentaje de superficie con: mercados, área v...",Estimaciones ONU-HABITAT en base a datos de Im...,A cada uno de los siguientes elementos se apli...,Manchas Urbanas
2,Ciudades inclusivas,Con el fin de lograr la integración de la ciud...,Reducir la desigualdad y la segregación socioe...,Proporción de personas que se sienten discrimi...,4,Porcentaje de población de 15 años y más edad ...,Encuesta de Hogares - EH 2019 del INE.,Porcentaje de la población de 15 años y más q...,Departamental - Urbano
3,Ciudades inclusivas,Con el fin de lograr la integración de la ciud...,Fortalecer las identidades culturales poniendo...,Eficiencia del gasto en promoción de cultura y...,5,Relación del presupuesto ejecutado respecto de...,"DGP Ministerio de Culturas y Turismo, reportes...",El monto de la inversión municipal en patrimo...,Departamental
4,Ciudades inclusivas,Con el fin de lograr la integración de la ciud...,Fortalecer las identidades culturales poniendo...,Participación en actividades culturales fuera ...,6,Porcentaje de personas de 18 años y más que pa...,Encuesta de Hogares - EH 2017 del INE.,Personas de 18 años y más edad que contestaron...,Departamental - Urbano
...,...,...,...,...,...,...,...,...,...
65,Ciudades productivas,Generar un entorno con mayores oportunidades p...,Elevar la productividad a partir de la gestión...,Tasa de ocupación,70,Porcentaje de ocupados respecto a la población...,Censo de Población y Vivienda del INE 2012.,Es la proporción que representa la Población O...,Manchas Urbanas
66,Ciudades productivas,Generar un entorno con mayores oportunidades p...,Elevar la productividad a partir de la gestión...,Tasa de informalidad en el empleo,71,Porcentaje de ocupados informales: trabajadore...,Estimaciones propias de ONU Hábitat en base a ...,Según la metodología de clasificación de los S...,Manchas Urbanas
67,Ciudades productivas,Generar un entorno con mayores oportunidades p...,Impulsar procesos de transformación productiva...,Proporción de unidades económicas de la indust...,72,Porcentaje de unidades económicas de la indust...,FUNDEMPRESA - Ministerio de Desarrollo Product...,El porcentaje de empresas manufactureras que c...,Total Bolivia
68,Ciudades productivas,Generar un entorno con mayores oportunidades p...,Impulsar procesos de transformación productiva...,Proporción de ocupados en actividades de innov...,73,"Porcentaje de ocupados como profesionales, cie...",Censo de Población y Vivienda del INE 2012. En...,Este indicador aproximado estima la capacidad ...,Manchas Urbanas


In [138]:
indicadores.to_csv('datos/indicadores_urbanos_catalogo.csv', index=False)

Valores para indicadores y variables

In [135]:
variables

,variable_codigo,periodo,unidad,nombre,indicador_codigo,indicador,municipio_id,municipio,valor
0,9,2019,[0-100],índice de Inequidad de Género (GII ) modificado,2,Índice modificado de Inequidad de Género,58,TOTALES AREA URBANA,9.862158
1,9,2019,[0-100],índice de Inequidad de Género (GII ) modificado,2,Índice modificado de Inequidad de Género,34,CHUQUISACA,9.981849
2,9,2019,[0-100],índice de Inequidad de Género (GII ) modificado,2,Índice modificado de Inequidad de Género,35,LA PAZ,9.869047
3,9,2019,[0-100],índice de Inequidad de Género (GII ) modificado,2,Índice modificado de Inequidad de Género,36,COCHABAMBA,8.246617
4,9,2019,[0-100],índice de Inequidad de Género (GII ) modificado,2,Índice modificado de Inequidad de Género,37,ORURO,13.319976
...,...,...,...,...,...,...,...,...,...
36,216,2012,Ocupados,Población Ocupada,73,Proporción de ocupados en actividades de innov...,41,BENI,144899.085300
37,216,2012,Ocupados,Población Ocupada,73,Proporción de ocupados en actividades de innov...,42,PANDO,41960.306950
0,221,2016,%,Unidades económicas de la industria manufactur...,74,Porcentaje de unidades económicas de la indust...,1,Total Bolivia,5.550445
0,220,2016,N° de empresas,Empresas manufactureras que declara haber real...,74,Porcentaje de unidades económicas de la indust...,1,Total Bolivia,362.000000


In [139]:
variables.to_csv('datos/indicadores_urbanos_variables.csv', index=False)

# Indicadores fiscales

In [142]:
def describir_indicador_fiscal(indicador_codigo):
    url = f'https://datosurbanosdebolivia.abe.bo/api/pndic/ind_desemp_fiscal/listar_indicador_unico/{indicador_codigo}'
    response = requests.get(url)
    i = response.json()['data']['indicadores'][0]
    c = response.json()['data']['componentes'][0]
    return {
        'descripcion': i['ind_descripcion'],
        'metodo': i['ind_metodo_calculo'],
        'fuente': i['ind_fuente_datos'],
        'unidad': c['uni_descripcion']
    }

In [162]:
def descargar_indicador_fiscal(indicador_codigo, indicador_nombre, indicador_unidad):
    url = f'https://datosurbanosdebolivia.abe.bo/api/pndic/ind_desemp_fiscal/listar_municipio/1/{indicador_codigo}'
    response = requests.get(url)
    datos = []
    municipios = response.json()['data']
    for municipio in municipios:
        valores = requests.get(f'https://datosurbanosdebolivia.abe.bo/api/pndic/ind_desemp_fiscal/gestion_indicador/{indicador_codigo}/{municipio["mun_id"]}')
        datos.extend([{'indicador_codigo':indicador_codigo, 'indicador':indicador_nombre, 'unidad':indicador_unidad, 'municipio_id':municipio["mun_id"], 'municipio': municipio["mun_detalle"], 'gestion': i['ges_gestion']['ges_descripcion'], 'valor': i['dig_valor']} for i in valores.json()['data']['indicadores']])
    return datos

El catálogo de indicadores fiscales:

In [146]:
url = 'https://datosurbanosdebolivia.abe.bo/api/pndic/ind_desemp_fiscal/tipo_dimension'
response = requests.get(url)
indicadores_fiscales = []

for dimension in response.json()['data']:
    for indicador in dimension['ind_indicador_desempenio_fiscals']:
        i = {
            'dimension': dimension['dim_descripcion'],
            'dimension_descripcion': dimension['dim_detalle'],
            'indicador': indicador['ind_detalle'],
            'indicador_codigo': indicador['ind_id']
        }
        metadata = describir_indicador_fiscal(i['indicador_codigo'])
        indicadores_fiscales.append({**i, **metadata})

In [172]:
indicadores_fiscales_catalogo = pd.DataFrame(indicadores_fiscales)
indicadores_fiscales_catalogo

,dimension,dimension_descripcion,indicador,indicador_codigo,descripcion,metodo,fuente,unidad
0,Ingreso,"La dimensión de Ingreso, se refiere a los indi...",Ingresos Totales,0,Ingresos Totales percibidos por el municipio e...,Evolución del total de los ingresos disponible...,Ministerio de Economía y Finanzas Públicas. Es...,Millones de Bs.
1,Ingreso,"La dimensión de Ingreso, se refiere a los indi...",Presupuesto por habitante,1,Indica la relación entre el presupuesto vigent...,PVH = Presupuesto Vigente año t / Población mu...,Ministerio de Economía y Finanzas Públicas. Es...,En Bs.
2,Ingreso,"La dimensión de Ingreso, se refiere a los indi...",Presupuesto por Km2,3,Indica la relación entre el presupuesto vigent...,PVKm2 = Presupuesto Vigente año t / Superficie...,Ministerio de Economía y Finanzas Públicas. Es...,En Bs.
3,Ingreso,"La dimensión de Ingreso, se refiere a los indi...",Presupuesto por área urbana (Ha),6,Se presentan el presupuesto vigente por superf...,PV/Ha= Presupuesto Vigente año t/Superficie ur...,Ministerio de Economía y Finanzas Públicas. Es...,En Bs.
4,Ingreso,"La dimensión de Ingreso, se refiere a los indi...",Autonomía financiera municipal,101,Refleja el peso relativo de los ingresos propi...,Ingresos propios dividido entre los ingresos t...,Ministerio de Economía y Finanzas Públicas. Pr...,Millones de Bs.
5,Ingreso,"La dimensión de Ingreso, se refiere a los indi...",Capacidad de Apalancamiento,103,Relación entre los ingresos obtenidos mediante...,AF=36000 Obtención de prestamos internos y de ...,Ministerio de Economía y Finanzas Públicas. Pr...,Bs
6,Ingreso,"La dimensión de Ingreso, se refiere a los indi...",Esfuerzo Fiscal Tributario,104,Indica la relación entre los ingresos tributar...,EFT=(13000 Ingresos Tributarios/19000 Transfer...,Ministerio de Economía y Finanzas Públicas. Pr...,Bs
7,Ingreso,"La dimensión de Ingreso, se refiere a los indi...",Dependencia de las transferencias y las regalías,105,La dependencia de las transferencias y las reg...,DTGN=(19000 Transferencias corrientes + 14000 ...,Ministerio de Economía y Finanzas Públicas. Pr...,Millones de Bs.
8,Ingreso,"La dimensión de Ingreso, se refiere a los indi...",Ingresos Basados en suelo respecto los ingreso...,106,Ingresos del gobierno municipal por concepto d...,IBSP=(13310 Impuesto a la propiedad de Bienes ...,Ministerio de Economía y Finanzas Públicas. Pr...,Bs
9,Ingreso,"La dimensión de Ingreso, se refiere a los indi...",Ingresos Basados en suelo respecto los ingreso...,107,Ingresos del gobierno municipal por concepto d...,IBSP=(13310 Impuesto a la propiedad de Bienes ...,Ministerio de Economía y Finanzas Públicas. Pr...,Millones de Bs.


In [176]:
indicadores_fiscales_catalogo.to_csv('datos/indicadores_fiscales_catalogo.csv', index=False)

Valores por municipio para cada indicador

In [163]:
datos = []
for indicador in tqdm(indicadores_fiscales, total=len(indicadores_fiscales), desc="indicadores"):
    datos.extend(descargar_indicador_fiscal(indicador['indicador_codigo'], indicador['indicador'], indicador['unidad']))

indicadores: 100%|██████████| 21/21 [39:54<00:00, 114.03s/it]


In [174]:
indicadores_fiscales_valores = pd.DataFrame(datos)

In [175]:
indicadores_fiscales_valores

,indicador_codigo,indicador,unidad,municipio_id,municipio,gestion,valor
0,0,Ingresos Totales,Millones de Bs.,1534,Acasio,2012,19.372409
1,0,Ingresos Totales,Millones de Bs.,1534,Acasio,2013,19.649940
2,0,Ingresos Totales,Millones de Bs.,1534,Acasio,2014,19.227833
3,0,Ingresos Totales,Millones de Bs.,1534,Acasio,2015,16.800811
4,0,Ingresos Totales,Millones de Bs.,1534,Acasio,2016,15.269259
...,...,...,...,...,...,...,...
41582,302,Acceso a información pública,None,1801,Trinidad,2012 - 2016 p,80.000000
41583,302,Acceso a información pública,None,1530,Uyuni,2012 - 2016 p,80.000000
41584,302,Acceso a información pública,None,1206,Viacha,2012 - 2016 p,70.000000
41585,302,Acceso a información pública,None,1537,Villazón,2012 - 2016 p,70.000000


In [177]:
indicadores_fiscales_valores.to_csv('datos/indicadores_fiscales.csv', index=False)

# Índice de Ciudades Prósperas

In [226]:
def consultar_metodo_cpi(indicador_codigo):
    url = f'https://datosurbanosdebolivia.abe.bo/api/cpi/ind_cpi/listar_indicador_id/{indicador_codigo}'
    response = requests.get(url)
    metodo = ' '.join([paso["mca_detalle"] if paso["mca_detalle"] else '' for paso in response.json()['data'][0]['mca_metodo_calculos']])
    fuentes = list(set([paso['mca_fuentes'] if paso['mca_fuentes'] else '' for paso in response.json()['data'][0]['mca_metodo_calculos']]))
    return metodo, ', '.join(fuentes)

In [211]:
def consultar_cpi(indicador_codigo, indicador_nombre):
    url = 'https://datosurbanosdebolivia.abe.bo/api/cpi/ind_cpi/indicador_valor_ciudad_id/7'
    response = requests.get(url)
    resultados = []
    for i in response.json()['data']:
        resultado = {
            'indicador': indicador_nombre,
            'area': i['aci_nombre'],
            'area_tipo': i['aci_tipo_area'],
            'area_id': i['aci_id'],
            'valor': i['rai_valor']
        }
        resultados.append(resultado)
    return resultados

In [220]:
def consultar_cpi_variables(indicador_codigo, indicador_nombre, indicador_valores):
    url = f'https://datosurbanosdebolivia.abe.bo/api/cpi/ind_cpi/cantidad_Variables_Id/{indicador_codigo}'
    response = requests.get(url)
    variables = {}
    for v in response.json()['data']:
        var = v['var_variables'][0]
        variables[var['var_id']] = {
            'variable': var['var_nombre'],
            'variable_descripcion': var['var_detalle'],
        }
    
    areas = {a['area_id']:{'area': a['area'], 'area_tipo': a['area_tipo']} for a in indicador_valores}
    response = requests.get(f'https://datosurbanosdebolivia.abe.bo/api/cpi/ind_cpi/cantidad_indicador_graficos/{indicador_codigo}')
    valores = []
    for i in response.json()['data']:
        valor = {
            'indicador': indicador_nombre,
            'variable': variables[i['var_id']]['variable'],
            'variable_descripcion': variables[i['var_id']]['variable_descripcion'],
            'area': areas[i['aci_id']]['area'],
            'area_tipo': areas[i['aci_id']]['area_tipo'],
            'valor': i['rav_valor']
        }
        valores.append(valor)
    return valores

In [192]:
url = 'https://datosurbanosdebolivia.abe.bo/api/cpi/ind_cpi/listar_dimencion_cpi'
response = requests.get(url)

In [227]:
cpi_catalogo = []
cpi_valores = []
cpi_variables = []
for dimension in tqdm(response.json()['data'], total=len(response.json()['data'])):
    for subdimension in dimension['sdim_sub_dimencions']:
        for indicador in subdimension['ind_indicadors']:
            i = {
                'dimension': dimension['dim_nombre'],
                'dimension_descripcion': dimension['dim_descripcion'],
                'subdimension': subdimension['sdim_nombre'],
                'subdimension_descripcion': subdimension['sdim_descripcion'],
                'indicador': indicador['ind_nombre'],
                'indicador_descripcion': indicador['ind_definicion'],
                'indicador_codigo': indicador['ind_id']
            }
            metodo, fuentes = consultar_metodo_cpi(i['indicador_codigo'])
            valor = consultar_cpi(i['indicador_codigo'], i['indicador'])
            variables = consultar_cpi_variables(i['indicador_codigo'], i['indicador'], valor)
            
            cpi_catalogo.append({**i, **{'metodo':metodo, 'fuentes':fuentes}})
            cpi_valores.extend(valor)
            cpi_variables.extend(variables)

100%|██████████| 6/6 [00:28<00:00,  4.68s/it]


El catálogo de componentes del índice de ciudades prósperas

In [238]:
ciudadesprosperas_catalogo = pd.DataFrame(cpi_catalogo)
ciudadesprosperas_catalogo

,dimension,dimension_descripcion,subdimension,subdimension_descripcion,indicador,indicador_descripcion,indicador_codigo,metodo,fuentes
0,Productividad,Con esta dimensión se quiere evaluar las condi...,Empleo,descripción subdimensión,Empleo informal,El empleo informal comprende todos los trabajo...,7,"Se utilizó la EH 2012, se calculó la población...","INE - CNPV 2012, Dossier UDAPE - 2018, INE - E..."
1,Productividad,Con esta dimensión se quiere evaluar las condi...,Empleo,descripción subdimensión,Tasa de desempleo,Personas que buscaron activamente empleo pero ...,5,Se clasifica a la población por Condición de A...,", INE - CNPV 2012"
2,Productividad,Con esta dimensión se quiere evaluar las condi...,Empleo,descripción subdimensión,Relación Empleo / Población,La Relación Empleo / Población es la proporció...,6,Se clasifica a la población por Condición de A...,INE - CNPV 2012
3,Productividad,Con esta dimensión se quiere evaluar las condi...,Fortaleza Económica,descripción subdimensión,Ingreso medio del hogar,El ingreso promedio del hogar incluye el ingre...,3,Se obtiene el ingreso mensual total de los hog...,", EPF 2016 - INE, Banco Mundial - 2016"
4,Productividad,Con esta dimensión se quiere evaluar las condi...,Fortaleza Económica,descripción subdimensión,Producto per cápita de la ciudad,El Producto per cápita de la Ciudad es la suma...,1,Se obtiene el PIB 2012 a precios corrientes po...,", INE - CNPV 2012, Banco Mundial (WB), INE - C..."
5,Productividad,Con esta dimensión se quiere evaluar las condi...,Fortaleza Económica,descripción subdimensión,Relación de dependencia -de la vejez-,La relación de dependencia de la vejez es la r...,2,Se obtienen los totales de población por grupo...,INE - CNPV 2012
6,Productividad,Con esta dimensión se quiere evaluar las condi...,Aglomeración económica,descripción subdimensión,Densidad económica,La densidad económica es el producto (PIB) de ...,4,Se tiene el valor del PIB en $us PPA - 2012 ob...,", Estimaciones ONU-HABITAT en base a datos de ..."
7,Desarrollo de Infraestructura,Esta dimensión pretende evaluar en términos cu...,Infraestructura social,descripción subdimensión,Densidad de médicos,"Número de médicos por cada 1.000 personas, en ...",14,Se toma en cuenta la población total del munic...,", https://bolivia.dotgiscorp.com/"
8,Desarrollo de Infraestructura,Esta dimensión pretende evaluar en términos cu...,Infraestructura social,descripción subdimensión,Número de bibliotecas públicas,Número de bibliotecas públicas por cada 100.00...,15,Se cuenta el número de Bibliotecas Públicas Lu...,", INE - CNPV 2012, PDTIs municipales, Open Str..."
9,Desarrollo de Infraestructura,Esta dimensión pretende evaluar en términos cu...,Tecnologías de la Información y la Comunicación,descripción subdimensión,Acceso a Internet,El acceso a Internet puede ser a través de una...,16,Se cuenta el número de hogares con conexión a ...,", INE - CNPV 2012"


In [233]:
ciudadesprosperas_catalogo.to_csv('datos/ciudadesprosperas_catalogo.csv', index=False)

Valores para cada componente

In [234]:
ciudadesprosperas_indicadores = pd.DataFrame(cpi_valores)
ciudadesprosperas_indicadores

,indicador,area,area_tipo,area_id,valor
0,Empleo informal,Área Metropolitana de La Paz,Metropolitana,1,5.4
1,Empleo informal,Área Metropolitana de Cochabamba,Metropolitana,2,12.2
2,Empleo informal,Área Metropolitana de Santa Cruz,Metropolitana,3,31.4
3,Empleo informal,Total 20 ciudades,Ciudad,4,16.6
4,Empleo informal,Sucre,Ciudad,5,4.8
...,...,...,...,...,...
1590,Eficiencia en el uso del suelo,Achocalla,Ciudad,25,0.0
1591,Eficiencia en el uso del suelo,Sipe Sipe,Ciudad,26,0.0
1592,Eficiencia en el uso del suelo,Vinto,Ciudad,27,4.9
1593,Eficiencia en el uso del suelo,Cotoca,Ciudad,28,25.2


In [235]:
ciudadesprosperas_indicadores.to_csv('datos/ciudadesprosperas_indicadores.csv', index=False)

Valores para variables que hacen a cada componente 

In [236]:
ciudadesprosperas_variables = pd.DataFrame(cpi_variables)
ciudadesprosperas_variables

,indicador,variable,variable_descripcion,area,area_tipo,valor
0,Empleo informal,Ocupados (sin empleadas del hogar) Municipio,Hab. 2012 Municipio,Área Metropolitana de La Paz,Metropolitana,785426.00
1,Empleo informal,Ocupados (sin empleadas del hogar) Municipio,Hab. 2012 Municipio,Área Metropolitana de Cochabamba,Metropolitana,494707.00
2,Empleo informal,Ocupados (sin empleadas del hogar) Municipio,Hab. 2012 Municipio,Área Metropolitana de Santa Cruz,Metropolitana,844158.00
3,Empleo informal,Ocupados (sin empleadas del hogar) Municipio,Hab. 2012 Municipio,Total 20 ciudades,Ciudad,2555340.00
4,Empleo informal,Ocupados (sin empleadas del hogar) Municipio,Hab. 2012 Municipio,Sucre,Ciudad,103335.00
...,...,...,...,...,...,...
3910,Eficiencia en el uso del suelo,Eficiencia uso tierra [crec sup/crec pop],na.,Achocalla,Ciudad,0.82
3911,Eficiencia en el uso del suelo,Eficiencia uso tierra [crec sup/crec pop],na.,Sipe Sipe,Ciudad,0.42
3912,Eficiencia en el uso del suelo,Eficiencia uso tierra [crec sup/crec pop],na.,Vinto,Ciudad,0.97
3913,Eficiencia en el uso del suelo,Eficiencia uso tierra [crec sup/crec pop],na.,Cotoca,Ciudad,3.22


In [237]:
ciudadesprosperas_variables.to_csv('datos/ciudadesprosperas_variables.csv', index=False)